In [1]:
!pip install fastparquet
!pip install seaborn 
!pip install scikit-learn
!pip install joblib

In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib
import xgboost as xgb
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

Tratamiento de datos archivo features

In [3]:
#Lectura Archivo features
features = pd.read_parquet('features.parquet')

In [4]:
#Visualización primeros 5 datos
pd.set_option('display.max_columns', None)
features

,frameTime,F0final_sma_stddev,F0final_sma_amean,voicingFinalUnclipped_sma_stddev,voicingFinalUnclipped_sma_amean,jitterLocal_sma_stddev,jitterLocal_sma_amean,jitterDDP_sma_stddev,jitterDDP_sma_amean,shimmerLocal_sma_stddev,shimmerLocal_sma_amean,logHNR_sma_stddev,logHNR_sma_amean,audspec_lengthL1norm_sma_stddev,audspec_lengthL1norm_sma_amean,audspecRasta_lengthL1norm_sma_stddev,audspecRasta_lengthL1norm_sma_amean,pcm_RMSenergy_sma_stddev,pcm_RMSenergy_sma_amean,pcm_zcr_sma_stddev,pcm_zcr_sma_amean,audSpec_Rfilt_sma[0]_stddev,audSpec_Rfilt_sma[0]_amean,audSpec_Rfilt_sma[1]_stddev,audSpec_Rfilt_sma[1]_amean,audSpec_Rfilt_sma[2]_stddev,audSpec_Rfilt_sma[2]_amean,audSpec_Rfilt_sma[3]_stddev,audSpec_Rfilt_sma[3]_amean,audSpec_Rfilt_sma[4]_stddev,audSpec_Rfilt_sma[4]_amean,audSpec_Rfilt_sma[5]_stddev,audSpec_Rfilt_sma[5]_amean,audSpec_Rfilt_sma[6]_stddev,audSpec_Rfilt_sma[6]_amean,audSpec_Rfilt_sma[7]_stddev,audSpec_Rfilt_sma[7]_amean,audSpec_Rfilt_sma[8]_stddev,audSpec_Rfilt_sma[8]_amean,audSpec_Rfilt_sma[9]_stddev,audSpec_Rfilt_sma[9]_amean,audSpec_Rfilt_sma[10]_stddev,audSpec_Rfilt_sma[10]_amean,audSpec_Rfilt_sma[11]_stddev,audSpec_Rfilt_sma[11]_amean,audSpec_Rfilt_sma[12]_stddev,audSpec_Rfilt_sma[12]_amean,audSpec_Rfilt_sma[13]_stddev,audSpec_Rfilt_sma[13]_amean,audSpec_Rfilt_sma[14]_stddev,audSpec_Rfilt_sma[14]_amean,audSpec_Rfilt_sma[15]_stddev,audSpec_Rfilt_sma[15]_amean,audSpec_Rfilt_sma[16]_stddev,audSpec_Rfilt_sma[16]_amean,audSpec_Rfilt_sma[17]_stddev,audSpec_Rfilt_sma[17]_amean,audSpec_Rfilt_sma[18]_stddev,audSpec_Rfilt_sma[18]_amean,audSpec_Rfilt_sma[19]_stddev,audSpec_Rfilt_sma[19]_amean,audSpec_Rfilt_sma[20]_stddev,audSpec_Rfilt_sma[20]_amean,audSpec_Rfilt_sma[21]_stddev,audSpec_Rfilt_sma[21]_amean,audSpec_Rfilt_sma[22]_stddev,audSpec_Rfilt_sma[22]_amean,audSpec_Rfilt_sma[23]_stddev,audSpec_Rfilt_sma[23]_amean,audSpec_Rfilt_sma[24]_stddev,audSpec_Rfilt_sma[24]_amean,audSpec_Rfilt_sma[25]_stddev,audSpec_Rfilt_sma[25]_amean,pcm_fftMag_fband250-650_sma_stddev,pcm_fftMag_fband250-650_sma_amean,pcm_fftMag_fband1000-4000_sma_stddev,pcm_fftMag_fband1000-4000_sma_amean,pcm_fftMag_spectralRollOff25.0_sma_stddev,pcm_fftMag_spectralRollOff25.0_sma_amean,pcm_fftMag_spectralRollOff50.0_sma_stddev,pcm_fftMag_spectralRollOff50.0_sma_amean,pcm_fftMag_spectralRollOff75.0_sma_stddev,pcm_fftMag_spectralRollOff75.0_sma_amean,pcm_fftMag_spectralRollOff90.0_sma_stddev,pcm_fftMag_spectralRollOff90.0_sma_amean,pcm_fftMag_spectralFlux_sma_stddev,pcm_fftMag_spectralFlux_sma_amean,pcm_fftMag_spectralCentroid_sma_stddev,pcm_fftMag_spectralCentroid_sma_amean,pcm_fftMag_spectralEntropy_sma_stddev,pcm_fftMag_spectralEntropy_sma_amean,pcm_fftMag_spectralVariance_sma_stddev,pcm_fftMag_spectralVariance_sma_amean,pcm_fftMag_spectralSkewness_sma_stddev,pcm_fftMag_spectralSkewness_sma_amean,pcm_fftMag_spectralKurtosis_sma_stddev,pcm_fftMag_spectralKurtosis_sma_amean,pcm_fftMag_spectralSlope_sma_stddev,pcm_fftMag_spectralSlope_sma_amean,pcm_fftMag_psySharpness_sma_stddev,pcm_fftMag_psySharpness_sma_amean,pcm_fftMag_spectralHarmonicity_sma_stddev,pcm_fftMag_spectralHarmonicity_sma_amean,pcm_fftMag_mfcc_sma[1]_stddev,pcm_fftMag_mfcc_sma[1]_amean,pcm_fftMag_mfcc_sma[2]_stddev,pcm_fftMag_mfcc_sma[2]_amean,pcm_fftMag_mfcc_sma[3]_stddev,pcm_fftMag_mfcc_sma[3]_amean,pcm_fftMag_mfcc_sma[4]_stddev,pcm_fftMag_mfcc_sma[4]_amean,pcm_fftMag_mfcc_sma[5]_stddev,pcm_fftMag_mfcc_sma[5]_amean,pcm_fftMag_mfcc_sma[6]_stddev,pcm_fftMag_mfcc_sma[6]_amean,pcm_fftMag_mfcc_sma[7]_stddev,pcm_fftMag_mfcc_sma[7]_amean,pcm_fftMag_mfcc_sma[8]_stddev,pcm_fftMag_mfcc_sma[8]_amean,pcm_fftMag_mfcc_sma[9]_stddev,pcm_fftMag_mfcc_sma[9]_amean,pcm_fftMag_mfcc_sma[10]_stddev,pcm_fftMag_mfcc_sma[10]_amean,pcm_fftMag_mfcc_sma[11]_stddev,pcm_fftMag_mfcc_sma[11]_amean,pcm_fftMag_mfcc_sma[12]_stddev,pcm_fftMag_mfcc_sma[12]_amean,pcm_fftMag_mfcc_sma[13]_stddev,pcm_fftMag_mfcc_sma[13]_amean,pcm_fftMag_mfcc_sma[14]_stddev,pcm_fftMag_mfcc_sma[14]_amean,F0final_sma_de_stddev,F0final_sma_de_amean,voicing

In [5]:
#Renombrar la columna numero_archivo por song_id
features = features.rename(columns={'numero_archivo': 'song_id'})
features.head()

,frameTime,F0final_sma_stddev,F0final_sma_amean,voicingFinalUnclipped_sma_stddev,voicingFinalUnclipped_sma_amean,jitterLocal_sma_stddev,jitterLocal_sma_amean,jitterDDP_sma_stddev,jitterDDP_sma_amean,shimmerLocal_sma_stddev,shimmerLocal_sma_amean,logHNR_sma_stddev,logHNR_sma_amean,audspec_lengthL1norm_sma_stddev,audspec_lengthL1norm_sma_amean,audspecRasta_lengthL1norm_sma_stddev,audspecRasta_lengthL1norm_sma_amean,pcm_RMSenergy_sma_stddev,pcm_RMSenergy_sma_amean,pcm_zcr_sma_stddev,pcm_zcr_sma_amean,audSpec_Rfilt_sma[0]_stddev,audSpec_Rfilt_sma[0]_amean,audSpec_Rfilt_sma[1]_stddev,audSpec_Rfilt_sma[1]_amean,audSpec_Rfilt_sma[2]_stddev,audSpec_Rfilt_sma[2]_amean,audSpec_Rfilt_sma[3]_stddev,audSpec_Rfilt_sma[3]_amean,audSpec_Rfilt_sma[4]_stddev,audSpec_Rfilt_sma[4]_amean,audSpec_Rfilt_sma[5]_stddev,audSpec_Rfilt_sma[5]_amean,audSpec_Rfilt_sma[6]_stddev,audSpec_Rfilt_sma[6]_amean,audSpec_Rfilt_sma[7]_stddev,audSpec_Rfilt_sma[7]_amean,audSpec_Rfilt_sma[8]_stddev,audSpec_Rfilt_sma[8]_amean,audSpec_Rfilt_sma[9]_stddev,audSpec_Rfilt_sma[9]_amean,audSpec_Rfilt_sma[10]_stddev,audSpec_Rfilt_sma[10]_amean,audSpec_Rfilt_sma[11]_stddev,audSpec_Rfilt_sma[11]_amean,audSpec_Rfilt_sma[12]_stddev,audSpec_Rfilt_sma[12]_amean,audSpec_Rfilt_sma[13]_stddev,audSpec_Rfilt_sma[13]_amean,audSpec_Rfilt_sma[14]_stddev,audSpec_Rfilt_sma[14]_amean,audSpec_Rfilt_sma[15]_stddev,audSpec_Rfilt_sma[15]_amean,audSpec_Rfilt_sma[16]_stddev,audSpec_Rfilt_sma[16]_amean,audSpec_Rfilt_sma[17]_stddev,audSpec_Rfilt_sma[17]_amean,audSpec_Rfilt_sma[18]_stddev,audSpec_Rfilt_sma[18]_amean,audSpec_Rfilt_sma[19]_stddev,audSpec_Rfilt_sma[19]_amean,audSpec_Rfilt_sma[20]_stddev,audSpec_Rfilt_sma[20]_amean,audSpec_Rfilt_sma[21]_stddev,audSpec_Rfilt_sma[21]_amean,audSpec_Rfilt_sma[22]_stddev,audSpec_Rfilt_sma[22]_amean,audSpec_Rfilt_sma[23]_stddev,audSpec_Rfilt_sma[23]_amean,audSpec_Rfilt_sma[24]_stddev,audSpec_Rfilt_sma[24]_amean,audSpec_Rfilt_sma[25]_stddev,audSpec_Rfilt_sma[25]_amean,pcm_fftMag_fband250-650_sma_stddev,pcm_fftMag_fband250-650_sma_amean,pcm_fftMag_fband1000-4000_sma_stddev,pcm_fftMag_fband1000-4000_sma_amean,pcm_fftMag_spectralRollOff25.0_sma_stddev,pcm_fftMag_spectralRollOff25.0_sma_amean,pcm_fftMag_spectralRollOff50.0_sma_stddev,pcm_fftMag_spectralRollOff50.0_sma_amean,pcm_fftMag_spectralRollOff75.0_sma_stddev,pcm_fftMag_spectralRollOff75.0_sma_amean,pcm_fftMag_spectralRollOff90.0_sma_stddev,pcm_fftMag_spectralRollOff90.0_sma_amean,pcm_fftMag_spectralFlux_sma_stddev,pcm_fftMag_spectralFlux_sma_amean,pcm_fftMag_spectralCentroid_sma_stddev,pcm_fftMag_spectralCentroid_sma_amean,pcm_fftMag_spectralEntropy_sma_stddev,pcm_fftMag_spectralEntropy_sma_amean,pcm_fftMag_spectralVariance_sma_stddev,pcm_fftMag_spectralVariance_sma_amean,pcm_fftMag_spectralSkewness_sma_stddev,pcm_fftMag_spectralSkewness_sma_amean,pcm_fftMag_spectralKurtosis_sma_stddev,pcm_fftMag_spectralKurtosis_sma_amean,pcm_fftMag_spectralSlope_sma_stddev,pcm_fftMag_spectralSlope_sma_amean,pcm_fftMag_psySharpness_sma_stddev,pcm_fftMag_psySharpness_sma_amean,pcm_fftMag_spectralHarmonicity_sma_stddev,pcm_fftMag_spectralHarmonicity_sma_amean,pcm_fftMag_mfcc_sma[1]_stddev,pcm_fftMag_mfcc_sma[1]_amean,pcm_fftMag_mfcc_sma[2]_stddev,pcm_fftMag_mfcc_sma[2]_amean,pcm_fftMag_mfcc_sma[3]_stddev,pcm_fftMag_mfcc_sma[3]_amean,pcm_fftMag_mfcc_sma[4]_stddev,pcm_fftMag_mfcc_sma[4]_amean,pcm_fftMag_mfcc_sma[5]_stddev,pcm_fftMag_mfcc_sma[5]_amean,pcm_fftMag_mfcc_sma[6]_stddev,pcm_fftMag_mfcc_sma[6]_amean,pcm_fftMag_mfcc_sma[7]_stddev,pcm_fftMag_mfcc_sma[7]_amean,pcm_fftMag_mfcc_sma[8]_stddev,pcm_fftMag_mfcc_sma[8]_amean,pcm_fftMag_mfcc_sma[9]_stddev,pcm_fftMag_mfcc_sma[9]_amean,pcm_fftMag_mfcc_sma[10]_stddev,pcm_fftMag_mfcc_sma[10]_amean,pcm_fftMag_mfcc_sma[11]_stddev,pcm_fftMag_mfcc_sma[11]_amean,pcm_fftMag_mfcc_sma[12]_stddev,pcm_fftMag_mfcc_sma[12]_amean,pcm_fftMag_mfcc_sma[13]_stddev,pcm_fftMag_mfcc_sma[13]_amean,pcm_fftMag_mfcc_sma[14]_stddev,pcm_fftMag_mfcc_sma[14]_amean,F0final_sma_de_stddev,F0final_sma_de_amean,voicing

Tratamiento de datos archivo arousal

In [6]:
#Lectura y visualización archivo arousal
df = pd.read_csv('arousal.csv')
df

song_id  sample_15000ms  sample_15500ms  sample_16000ms  sample_16500ms  \
0           2       -0.109386       -0.114942   -1.164126e-01       -0.118613   
1           3       -0.110846       -0.123973   -1.311026e-01       -0.135956   
2           4        0.222327        0.179446    1.783881e-01        0.184056   
3           5       -0.255613       -0.251579   -2.519577e-01       -0.251124   
4           7        0.464234        0.460789    4.609907e-01        0.461046   
...       ...             ...             ...             ...             ...   
1797     2054       -0.190000       -0.182000   -1.800000e-01       -0.158000   
1798     2055       -0.070000       -0.086000   -1.100000e-01       -0.072000   
1799     2056       -0.044000       -0.044000   -4.400000e-02       -0.044000   
1800     2057        0.413333        0.413333    4.116667e-01        0.410000   
1801     2058        0.008000        0.002000   -2.775558e-18        0.006000   

      sample_17000ms  sample_17500ms  sample_18000ms  sample_18500ms  \
0      -1.264569e-01       -0.133199       -0.136855       -0.144713   
1      -1.407755e-01       -0.144664       -0.163118       -0.165218   
2       1.760421e-01        0.178720        0.176345        0.175793   
3      -2.507627e-01       -0.251957       -0.251957       -0.251957   
4       4.572397e-01        0.465702        0.471809        0.469918   
...              ...             ...             ...             ...   
1797   -1.560000e-01       -0.150000       -0.152000       -0.150000   
1798   -6.200000e-02       -0.062000       -0.076000       -0.062000   
1799   -4.200000e-02       -0.046000       -0.050000       -0.052000   
1800    4.066667e-01        0.410000        0.401667        0.371667   
1801   -5.551115e-18        0.002000        0.002000        0.002000   

      sample_19000ms  sample_19500ms  sample_20000ms  sample_20500ms  \
0          -0.138985   -1.376024e-01       -0.138575       -0.140522   
1          -0.158858   -1.567828e-01       -0.160081       -0.169374   
2           0.176154    1.766041e-01        0.180138        0.176360   
3          -0.251957   -2.519567e-01       -0.250848       -0.247444   
4           0.473377    4.700510e-01        0.464498        0.471905   
...              ...             ...             ...             ...   
1797       -0.150000   -1.460000e-01       -0.152000       -0.154000   
1798       -0.042000   -4.600000e-02       -0.018000       -0.024000   
1799       -0.052000   -5.400000e-02       -0.054000       -0.056000   
1800        0.340000    3.283333e-01        0.328333        0.338333   
1801        0.006000   -5.551115e-18       -0.030000       -0.070000   

      sample_21000ms  sample_21500ms  sample_22000ms  sample_22500ms  \
0          -0.144348       -0.150365       -0.152446   -1.534950e-01   
1          -0.165857       -0.164108       -0.161764   -1.624463e-01   
2           0.168876        0.171422        0.169539    1.684931e-01   
3          -0.242066       -0.241087       -0.241087   -2.410867e-01   
4           0.477970        0.449070        0.403073    3.940040e-01   
...              ...             ...             ...             ...   
1797       -0.158000       -0.164000       -0.166000   -1.760000e-01   
1798       -0.014000       -0.012000       -0.008000   -4.163336e-18   
1799       -0.060000       -0.060000       -0.060000   -6.000000e-02   
1800        0.318333        0.301667        0.291667    2.783333e-01   
1801       -0.086000        0.006000        0.068000    8.400000e-02   

      sample_23000ms  sample_23500ms  sample_24000ms  sample_24500ms  \
0          -0.155344       -0.153442       -0.147347       -0.146429   
1          -0.170079       -0.168090       -0.164870       -0.161530   
2           0.169783        0.173216        0.164226        0.165260   
3          -0.238962       -0.239783       -0.237609       -0.237609   
4           0.398359        0.393433        0.392229        0.386149   
...   

In [7]:
# Mantener todas las columnas hasta sample_44500ms (inclusive)
arousal = df.loc[:, :'sample_44500ms']
arousal.head()

,song_id,sample_15000ms,sample_15500ms,sample_16000ms,sample_16500ms,sample_17000ms,sample_17500ms,sample_18000ms,sample_18500ms,sample_19000ms,sample_19500ms,sample_20000ms,sample_20500ms,sample_21000ms,sample_21500ms,sample_22000ms,sample_22500ms,sample_23000ms,sample_23500ms,sample_24000ms,sample_24500ms,sample_25000ms,sample_25500ms,sample_26000ms,sample_26500ms,sample_27000ms,sample_27500ms,sample_28000ms,sample_28500ms,sample_29000ms,sample_29500ms,sample_30000ms,sample_30500ms,sample_31000ms,sample_31500ms,sample_32000ms,sample_32500ms,sample_33000ms,sample_33500ms,sample_34000ms,sample_34500ms,sample_35000ms,sample_35500ms,sample_36000ms,sample_36500ms,sample_37000ms,sample_37500ms,sample_38000ms,sample_38500ms,sample_39000ms,sample_39500ms,sample_40000ms,sample_40500ms,sample_41000ms,sample_41500ms,sample_42000ms,sample_42500ms,sample_43000ms,sample_43500ms,sample_44000ms,sample_44500ms
0,2,-0.109386,-0.114942,-0.116413,-0.118613,-0.126457,-0.133199,-0.136855,-0.144713,-0.138985,-0.137602,-0.138575,-0.140522,-0.144348,-0.150365,-0.152446,-0.153495,-0.155344,-0.153442,-0.147347,-0.146429,-0.143975,-0.141597,-0.140725,-0.140686,-0.139947,-0.147282,-0.157786,-0.164249,-0.168293,-0.164517,-0.157689,-0.153775,-0.207365,-0.253384,-0.256741,-0.254217,-0.252781,-0.258112,-0.263372,-0.270176,-0.272033,-0.267793,-0.262615,-0.261571,-0.261160,-0.258945,-0.262110,-0.261051,-0.256467,-0.258118,-0.256900,-0.259548,-0.262236,-0.261480,-0.264915,-0.265395,-0.265161,-0.264469,-0.265841,-0.267091
1,3,-0.110846,-0.123973,-0.131103,-0.135956,-0.140775,-0.144664,-0.163118,-0.165218,-0.158858,-0.156783,-0.160081,-0.169374,-0.165857,-0.164108,-0.161764,-0.162446,-0.170079,-0.168090,-0.164870,-0.161530,-0.161725,-0.171526,-0.172795,-0.172407,-0.169303,-0.172701,-0.174472,-0.168315,-0.166760,-0.164218,-0.164253,-0.174865,-0.181806,-0.181193,-0.185401,-0.183284,-0.195656,-0.204603,-0.201497,-0.200207,-0.199741,-0.215122,-0.223772,-0.229498,-0.235222,-0.241272,-0.246206,-0.253427,-0.256365,-0.261529,-0.265677,-0.265931,-0.263960,-0.263572,-0.262956,-0.260338,-0.248711,-0.245517,-0.251300,-0.254620
2,4,0.222327,0.179446,0.178388,0.184056,0.176042,0.178720,0.176345,0.175793,0.176154,0.176604,0.180138,0.176360,0.168876,0.171422,0.169539,0.168493,0.169783,0.173216,0.164226,0.165260,0.172209,0.177908,0.185147,0.215622,0.256678,0.263116,0.246703,0.243829,0.248545,0.249931,0.262921,0.260946,0.271850,0.265925,0.274905,0.270707,0.282363,0.281385,0.282713,0.278853,0.302644,0.295503,0.295632,0.294176,0.307456,0.315198,0.309677,0.307304,0.306245,0.309408,0.309623,0.298644,0.294143,0.302702,0.297487,0.303423,0.298097,0.304339,0.304114,0.305078
3,5,-0.255613,-0.251579,-0.251958,-0.251124,-0.250763,-0.251957,-0.251957,-0.251957,-0.251957,-0.251957,-0.250848,-0.247444,-0.242066,-0.241087,-0.241087,-0.241087,-0.238962,-0.239783,-0.237609,-0.237609,-0.237609,-0.237609,-0.236711,-0.235870,-0.235870,-0.235870,-0.234921,-0.232773,-0.230218,-0.230182,-0.226740,-0.226740,-0.227618,-0.228476,-0.226738,-0.224566,-0.222392,-0.220232,-0.224296,-0.234855,-0.238744,-0.239349,-0.238913,-0.231930,-0.232801,-0.232827,-0.232012,-0.230246,-0.229089,-0.224347,-0.222827,-0.225900,-0.228914,-0.228914,-0.228914,-0.228914,-0.227562,-0.227609,-0.229997,-0.243921
4,7,0.464234,0.460789,0.460991,0.461046,0.457240,0.465702,0.471809,0.469918,0.473377,0.470051,0.464498,0.471905,0.477970,0.449070,0.403073,0.394004,0.398359,0.393433,0.392229,0.386149,0.385554,0.384041,0.385499,0.381375,0.377777,0.370106,0.346994,0.349432,0.351037,0.342570,0.343068,0.346314,0.341944,0.339297,0.336535,0.331994,0.332121,0.333515,0.326968,0.325432,0.326778,0.326238,0.324213,0.329854,0.330195,0.331782,0.331161,0.331736,0.332561,0.329794,0.331689,0.333163,0.331243,0.333925,0.333738,0.340781,0.339002,0.339272,0.340961,0.342020


In [8]:
#Transponer columnas sample

# Identificar las columnas de sample
sample_columns = [col for col in arousal.columns if col.startswith("sample_")]

# Usar melt para transponer las columnas de sample en una sola columna llamada "arousal"
df_melted = arousal.melt(id_vars=["song_id"],
                    value_vars=sample_columns,
                    var_name="sample",
                    value_name="arousal")

# Extraer solo el número del sample (por ejemplo, de "sample_1500" a "1500")
df_melted["sample"] = df_melted["sample"].str.extract(r'(\d+)').astype(int)

# Ordenar por "song_id" y luego por "sample" para asegurar que los datos estén bien organizados
df_melted_arousal = df_melted.sort_values(by=["song_id", "sample"]).reset_index(drop=True)

# Mostrar el resultado para verificar


In [9]:
#Visualización del dataset transpuesto
df_melted_arousal

,song_id,sample,arousal
0,2,15000,-0.109386
1,2,15500,-0.114942
2,2,16000,-0.116413
3,2,16500,-0.118613
4,2,17000,-0.126457
...,...,...,...
108115,2058,42500,-0.016000
108116,2058,43000,0.018000
108117,2058,43500,0.054000
108118,2058,44000,0.070000


In [10]:
#Se realiza el merge del dataframe df_limpio y arousal
df_farousal = pd.merge(features, df_melted_arousal, on='song_id', how='inner')
df_farousal

,frameTime,F0final_sma_stddev,F0final_sma_amean,voicingFinalUnclipped_sma_stddev,voicingFinalUnclipped_sma_amean,jitterLocal_sma_stddev,jitterLocal_sma_amean,jitterDDP_sma_stddev,jitterDDP_sma_amean,shimmerLocal_sma_stddev,shimmerLocal_sma_amean,logHNR_sma_stddev,logHNR_sma_amean,audspec_lengthL1norm_sma_stddev,audspec_lengthL1norm_sma_amean,audspecRasta_lengthL1norm_sma_stddev,audspecRasta_lengthL1norm_sma_amean,pcm_RMSenergy_sma_stddev,pcm_RMSenergy_sma_amean,pcm_zcr_sma_stddev,pcm_zcr_sma_amean,audSpec_Rfilt_sma[0]_stddev,audSpec_Rfilt_sma[0]_amean,audSpec_Rfilt_sma[1]_stddev,audSpec_Rfilt_sma[1]_amean,audSpec_Rfilt_sma[2]_stddev,audSpec_Rfilt_sma[2]_amean,audSpec_Rfilt_sma[3]_stddev,audSpec_Rfilt_sma[3]_amean,audSpec_Rfilt_sma[4]_stddev,audSpec_Rfilt_sma[4]_amean,audSpec_Rfilt_sma[5]_stddev,audSpec_Rfilt_sma[5]_amean,audSpec_Rfilt_sma[6]_stddev,audSpec_Rfilt_sma[6]_amean,audSpec_Rfilt_sma[7]_stddev,audSpec_Rfilt_sma[7]_amean,audSpec_Rfilt_sma[8]_stddev,audSpec_Rfilt_sma[8]_amean,audSpec_Rfilt_sma[9]_stddev,audSpec_Rfilt_sma[9]_amean,audSpec_Rfilt_sma[10]_stddev,audSpec_Rfilt_sma[10]_amean,audSpec_Rfilt_sma[11]_stddev,audSpec_Rfilt_sma[11]_amean,audSpec_Rfilt_sma[12]_stddev,audSpec_Rfilt_sma[12]_amean,audSpec_Rfilt_sma[13]_stddev,audSpec_Rfilt_sma[13]_amean,audSpec_Rfilt_sma[14]_stddev,audSpec_Rfilt_sma[14]_amean,audSpec_Rfilt_sma[15]_stddev,audSpec_Rfilt_sma[15]_amean,audSpec_Rfilt_sma[16]_stddev,audSpec_Rfilt_sma[16]_amean,audSpec_Rfilt_sma[17]_stddev,audSpec_Rfilt_sma[17]_amean,audSpec_Rfilt_sma[18]_stddev,audSpec_Rfilt_sma[18]_amean,audSpec_Rfilt_sma[19]_stddev,audSpec_Rfilt_sma[19]_amean,audSpec_Rfilt_sma[20]_stddev,audSpec_Rfilt_sma[20]_amean,audSpec_Rfilt_sma[21]_stddev,audSpec_Rfilt_sma[21]_amean,audSpec_Rfilt_sma[22]_stddev,audSpec_Rfilt_sma[22]_amean,audSpec_Rfilt_sma[23]_stddev,audSpec_Rfilt_sma[23]_amean,audSpec_Rfilt_sma[24]_stddev,audSpec_Rfilt_sma[24]_amean,audSpec_Rfilt_sma[25]_stddev,audSpec_Rfilt_sma[25]_amean,pcm_fftMag_fband250-650_sma_stddev,pcm_fftMag_fband250-650_sma_amean,pcm_fftMag_fband1000-4000_sma_stddev,pcm_fftMag_fband1000-4000_sma_amean,pcm_fftMag_spectralRollOff25.0_sma_stddev,pcm_fftMag_spectralRollOff25.0_sma_amean,pcm_fftMag_spectralRollOff50.0_sma_stddev,pcm_fftMag_spectralRollOff50.0_sma_amean,pcm_fftMag_spectralRollOff75.0_sma_stddev,pcm_fftMag_spectralRollOff75.0_sma_amean,pcm_fftMag_spectralRollOff90.0_sma_stddev,pcm_fftMag_spectralRollOff90.0_sma_amean,pcm_fftMag_spectralFlux_sma_stddev,pcm_fftMag_spectralFlux_sma_amean,pcm_fftMag_spectralCentroid_sma_stddev,pcm_fftMag_spectralCentroid_sma_amean,pcm_fftMag_spectralEntropy_sma_stddev,pcm_fftMag_spectralEntropy_sma_amean,pcm_fftMag_spectralVariance_sma_stddev,pcm_fftMag_spectralVariance_sma_amean,pcm_fftMag_spectralSkewness_sma_stddev,pcm_fftMag_spectralSkewness_sma_amean,pcm_fftMag_spectralKurtosis_sma_stddev,pcm_fftMag_spectralKurtosis_sma_amean,pcm_fftMag_spectralSlope_sma_stddev,pcm_fftMag_spectralSlope_sma_amean,pcm_fftMag_psySharpness_sma_stddev,pcm_fftMag_psySharpness_sma_amean,pcm_fftMag_spectralHarmonicity_sma_stddev,pcm_fftMag_spectralHarmonicity_sma_amean,pcm_fftMag_mfcc_sma[1]_stddev,pcm_fftMag_mfcc_sma[1]_amean,pcm_fftMag_mfcc_sma[2]_stddev,pcm_fftMag_mfcc_sma[2]_amean,pcm_fftMag_mfcc_sma[3]_stddev,pcm_fftMag_mfcc_sma[3]_amean,pcm_fftMag_mfcc_sma[4]_stddev,pcm_fftMag_mfcc_sma[4]_amean,pcm_fftMag_mfcc_sma[5]_stddev,pcm_fftMag_mfcc_sma[5]_amean,pcm_fftMag_mfcc_sma[6]_stddev,pcm_fftMag_mfcc_sma[6]_amean,pcm_fftMag_mfcc_sma[7]_stddev,pcm_fftMag_mfcc_sma[7]_amean,pcm_fftMag_mfcc_sma[8]_stddev,pcm_fftMag_mfcc_sma[8]_amean,pcm_fftMag_mfcc_sma[9]_stddev,pcm_fftMag_mfcc_sma[9]_amean,pcm_fftMag_mfcc_sma[10]_stddev,pcm_fftMag_mfcc_sma[10]_amean,pcm_fftMag_mfcc_sma[11]_stddev,pcm_fftMag_mfcc_sma[11]_amean,pcm_fftMag_mfcc_sma[12]_stddev,pcm_fftMag_mfcc_sma[12]_amean,pcm_fftMag_mfcc_sma[13]_stddev,pcm_fftMag_mfcc_sma[13]_amean,pcm_fftMag_mfcc_sma[14]_stddev,pcm_fftMag_mfcc_sma[14]_amean,F0final_sma_de_stddev,F0final_sma_de_amean,voicing

Tratamiento datos dataset features y arousal

In [11]:
# Tomamos la muestra para hacer el análisis manejable
sample_size = 100000
df_s_arousal = df_farousal.sample(n=sample_size, random_state=42)

In [12]:
df_s_arousal

,frameTime,F0final_sma_stddev,F0final_sma_amean,voicingFinalUnclipped_sma_stddev,voicingFinalUnclipped_sma_amean,jitterLocal_sma_stddev,jitterLocal_sma_amean,jitterDDP_sma_stddev,jitterDDP_sma_amean,shimmerLocal_sma_stddev,shimmerLocal_sma_amean,logHNR_sma_stddev,logHNR_sma_amean,audspec_lengthL1norm_sma_stddev,audspec_lengthL1norm_sma_amean,audspecRasta_lengthL1norm_sma_stddev,audspecRasta_lengthL1norm_sma_amean,pcm_RMSenergy_sma_stddev,pcm_RMSenergy_sma_amean,pcm_zcr_sma_stddev,pcm_zcr_sma_amean,audSpec_Rfilt_sma[0]_stddev,audSpec_Rfilt_sma[0]_amean,audSpec_Rfilt_sma[1]_stddev,audSpec_Rfilt_sma[1]_amean,audSpec_Rfilt_sma[2]_stddev,audSpec_Rfilt_sma[2]_amean,audSpec_Rfilt_sma[3]_stddev,audSpec_Rfilt_sma[3]_amean,audSpec_Rfilt_sma[4]_stddev,audSpec_Rfilt_sma[4]_amean,audSpec_Rfilt_sma[5]_stddev,audSpec_Rfilt_sma[5]_amean,audSpec_Rfilt_sma[6]_stddev,audSpec_Rfilt_sma[6]_amean,audSpec_Rfilt_sma[7]_stddev,audSpec_Rfilt_sma[7]_amean,audSpec_Rfilt_sma[8]_stddev,audSpec_Rfilt_sma[8]_amean,audSpec_Rfilt_sma[9]_stddev,audSpec_Rfilt_sma[9]_amean,audSpec_Rfilt_sma[10]_stddev,audSpec_Rfilt_sma[10]_amean,audSpec_Rfilt_sma[11]_stddev,audSpec_Rfilt_sma[11]_amean,audSpec_Rfilt_sma[12]_stddev,audSpec_Rfilt_sma[12]_amean,audSpec_Rfilt_sma[13]_stddev,audSpec_Rfilt_sma[13]_amean,audSpec_Rfilt_sma[14]_stddev,audSpec_Rfilt_sma[14]_amean,audSpec_Rfilt_sma[15]_stddev,audSpec_Rfilt_sma[15]_amean,audSpec_Rfilt_sma[16]_stddev,audSpec_Rfilt_sma[16]_amean,audSpec_Rfilt_sma[17]_stddev,audSpec_Rfilt_sma[17]_amean,audSpec_Rfilt_sma[18]_stddev,audSpec_Rfilt_sma[18]_amean,audSpec_Rfilt_sma[19]_stddev,audSpec_Rfilt_sma[19]_amean,audSpec_Rfilt_sma[20]_stddev,audSpec_Rfilt_sma[20]_amean,audSpec_Rfilt_sma[21]_stddev,audSpec_Rfilt_sma[21]_amean,audSpec_Rfilt_sma[22]_stddev,audSpec_Rfilt_sma[22]_amean,audSpec_Rfilt_sma[23]_stddev,audSpec_Rfilt_sma[23]_amean,audSpec_Rfilt_sma[24]_stddev,audSpec_Rfilt_sma[24]_amean,audSpec_Rfilt_sma[25]_stddev,audSpec_Rfilt_sma[25]_amean,pcm_fftMag_fband250-650_sma_stddev,pcm_fftMag_fband250-650_sma_amean,pcm_fftMag_fband1000-4000_sma_stddev,pcm_fftMag_fband1000-4000_sma_amean,pcm_fftMag_spectralRollOff25.0_sma_stddev,pcm_fftMag_spectralRollOff25.0_sma_amean,pcm_fftMag_spectralRollOff50.0_sma_stddev,pcm_fftMag_spectralRollOff50.0_sma_amean,pcm_fftMag_spectralRollOff75.0_sma_stddev,pcm_fftMag_spectralRollOff75.0_sma_amean,pcm_fftMag_spectralRollOff90.0_sma_stddev,pcm_fftMag_spectralRollOff90.0_sma_amean,pcm_fftMag_spectralFlux_sma_stddev,pcm_fftMag_spectralFlux_sma_amean,pcm_fftMag_spectralCentroid_sma_stddev,pcm_fftMag_spectralCentroid_sma_amean,pcm_fftMag_spectralEntropy_sma_stddev,pcm_fftMag_spectralEntropy_sma_amean,pcm_fftMag_spectralVariance_sma_stddev,pcm_fftMag_spectralVariance_sma_amean,pcm_fftMag_spectralSkewness_sma_stddev,pcm_fftMag_spectralSkewness_sma_amean,pcm_fftMag_spectralKurtosis_sma_stddev,pcm_fftMag_spectralKurtosis_sma_amean,pcm_fftMag_spectralSlope_sma_stddev,pcm_fftMag_spectralSlope_sma_amean,pcm_fftMag_psySharpness_sma_stddev,pcm_fftMag_psySharpness_sma_amean,pcm_fftMag_spectralHarmonicity_sma_stddev,pcm_fftMag_spectralHarmonicity_sma_amean,pcm_fftMag_mfcc_sma[1]_stddev,pcm_fftMag_mfcc_sma[1]_amean,pcm_fftMag_mfcc_sma[2]_stddev,pcm_fftMag_mfcc_sma[2]_amean,pcm_fftMag_mfcc_sma[3]_stddev,pcm_fftMag_mfcc_sma[3]_amean,pcm_fftMag_mfcc_sma[4]_stddev,pcm_fftMag_mfcc_sma[4]_amean,pcm_fftMag_mfcc_sma[5]_stddev,pcm_fftMag_mfcc_sma[5]_amean,pcm_fftMag_mfcc_sma[6]_stddev,pcm_fftMag_mfcc_sma[6]_amean,pcm_fftMag_mfcc_sma[7]_stddev,pcm_fftMag_mfcc_sma[7]_amean,pcm_fftMag_mfcc_sma[8]_stddev,pcm_fftMag_mfcc_sma[8]_amean,pcm_fftMag_mfcc_sma[9]_stddev,pcm_fftMag_mfcc_sma[9]_amean,pcm_fftMag_mfcc_sma[10]_stddev,pcm_fftMag_mfcc_sma[10]_amean,pcm_fftMag_mfcc_sma[11]_stddev,pcm_fftMag_mfcc_sma[11]_amean,pcm_fftMag_mfcc_sma[12]_stddev,pcm_fftMag_mfcc_sma[12]_amean,pcm_fftMag_mfcc_sma[13]_stddev,pcm_fftMag_mfcc_sma[13]_amean,pcm_fftMag_mfcc_sma[14]_stddev,pcm_fftMag_mfcc_sma[14]_amean,F0final_sma_de_stddev,F0final_sma_de_amean,voicing

In [13]:
def analizar_caracteristicas(X, y_arousal, umbral_varianza=0.95):
    # Estandarizar características
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Análisis PCA para determinar número óptimo de componentes
    pca = PCA()
    pca.fit(X_scaled)
    
    # Calcular varianza acumulada
    varianza_acumulada = np.cumsum(pca.explained_variance_ratio_)
    n_componentes = np.argmax(varianza_acumulada >= umbral_varianza) + 1
    
    # Selección usando f_regression
    selector = SelectKBest(score_func=f_regression)
    selector.fit(X_scaled, y_arousal)
    
    # Calcular importancia mutua
    mi_scores = mutual_info_regression(X_scaled, y_arousal)
    
    # Combinar resultados
    resultados = pd.DataFrame({
        'Característica': X.columns,
        'F_Score': selector.scores_,
        'MI_Score': mi_scores
    })
    
    # Normalizar scores
    resultados['F_Score_Norm'] = (resultados['F_Score'] - resultados['F_Score'].min()) / (resultados['F_Score'].max() - resultados['F_Score'].min())
    resultados['MI_Score_Norm'] = (resultados['MI_Score'] - resultados['MI_Score'].min()) / (resultados['MI_Score'].max() - resultados['MI_Score'].min())
    
    # Score combinado
    resultados['Score_Combinado'] = (resultados['F_Score_Norm'] + resultados['MI_Score_Norm']) / 2
    
    # Ordenar por score combinado
    resultados_ordenados = resultados.sort_values('Score_Combinado', ascending=False)
    
    # Determinar número de características basado en PCA
    caracteristicas_seleccionadas = resultados_ordenados.head(n_componentes)
    
    return caracteristicas_seleccionadas, n_componentes, varianza_acumulada

def evaluar_modelo(X, y_arousal, caracteristicas_seleccionadas):
    # Usar solo las características seleccionadas
    X_reducido = X[caracteristicas_seleccionadas['Característica']]
    
    # Split y_arousal entrenamiento
    X_train, X_test, y_train, y_test = train_test_split(X_reducido, y_arousal, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Evaluar
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    return r2, rmse, model

# Ejemplo de uso
# df = tu_dataframe
X = df_s_arousal.drop(['frameTime','song_id','sample'], axis=1)  # Ajusta según tus columnas
y_arousal = X['arousal'] if 'arousal' in X.columns else None
X = X.drop(['arousal'], axis=1)

# Análisis de características
umbral_varianza = 0.95
caracteristicas, n_componentes, varianza = analizar_caracteristicas(X, y_arousal, umbral_varianza)
print(f"Número óptimo de características para mantener {umbral_varianza*100}% de la varianza: {n_componentes}")
print(f"Número óptimo de características para mantener {umbral_varianza*100}% de la varianza: {n_componentes}")

# Evaluación del modelo
r2, rmse, modelo = evaluar_modelo(X, y_arousal, caracteristicas)
print(f"\nRendimiento del modelo:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")

# Mostrar top características
print("\nTop características seleccionadas:")
print(caracteristicas[['Característica', 'Score_Combinado']])

Número óptimo de características para mantener 95.0% de la varianza: 87
Número óptimo de características para mantener 95.0% de la varianza: 87

Rendimiento del modelo:
R² Score: 0.5115
RMSE: 0.2005

Top características seleccionadas:
                                  Característica  Score_Combinado
103                 pcm_fftMag_mfcc_sma[1]_amean         1.000000
13                audspec_lengthL1norm_sma_amean         0.668488
91         pcm_fftMag_spectralVariance_sma_amean         0.620631
19                             pcm_zcr_sma_amean         0.556377
10                             logHNR_sma_stddev         0.536694
..                                           ...              ...
123                pcm_fftMag_mfcc_sma[11]_amean         0.126857
76     pcm_fftMag_spectralRollOff25.0_sma_stddev         0.126561
73             pcm_fftMag_fband250-650_sma_amean         0.124777
213  pcm_fftMag_spectralRollOff90.0_sma_de_amean         0.123676
16                      pcm_RMSenergy_s

In [14]:

def entrenar_evaluar_modelos(X, y_arousal, caracteristicas_seleccionadas, ruta_guardado='mejor_modelo.joblib'):
    # Usar características seleccionadas
    X_reducido = X[caracteristicas_seleccionadas['Característica']]
    
    # Split de datos
    X_train, X_test, y_train, y_test = train_test_split(
        X_reducido, y_arousal, test_size=0.2, random_state=42
    )
    
    # Escalado
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Guardar el scaler para uso futuro
    joblib.dump(scaler, 'scaler.joblib')
    
    # Definir modelos
    modelos = {
        'Ridge': Ridge(),
        'Lasso': Lasso(),
        'ElasticNet': ElasticNet(),
        'RandomForest': RandomForestRegressor(),
        'GradientBoosting': GradientBoostingRegressor(),
        'XGBoost': xgb.XGBRegressor()
    }
    
    # Parámetros para GridSearch
    parametros = {
        'Ridge': {'alpha': [0.1, 1.0, 10.0]},
        'Lasso': {'alpha': [0.1, 1.0, 10.0]},
        'ElasticNet': {'alpha': [0.1, 1.0], 'l1_ratio': [0.2, 0.5, 0.8]},
        'RandomForest': {
            'n_estimators': [100, 200],
            'max_depth': [10, 20],
            'min_samples_split': [2, 5]
        },
        'GradientBoosting': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5]
        },
        'XGBoost': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5]
        }
    }
    
    resultados = {}
    mejor_score = float('-inf')
    mejor_modelo_final = None
    
    for nombre, modelo in modelos.items():
        # GridSearch con validación cruzada
        grid = GridSearchCV(
            modelo, 
            parametros[nombre], 
            cv=3, 
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        
        grid.fit(X_train_scaled, y_train)
        
        # Predicciones
        y_pred = grid.predict(X_test_scaled)
        
        # Métricas
        r2 = r2_score(y_test, y_pred)
        resultados[nombre] = {
            'R2': r2,
            'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
            'MAE': mean_absolute_error(y_test, y_pred),
            'Mejores_Parametros': grid.best_params_
        }
        
        # Actualizar mejor modelo basado en R2
        if r2 > mejor_score:
            mejor_score = r2
            mejor_modelo_final = grid.best_estimator_
            mejor_nombre = nombre
    
    # Guardar el mejor modelo
    joblib.dump(mejor_modelo_final, ruta_guardado)
    
    # Guardar información sobre las características seleccionadas
    joblib.dump(caracteristicas_seleccionadas['Característica'].tolist(), 'caracteristicas_seleccionadas.joblib')
    
    print(f"\nMejor modelo: {mejor_nombre} (R2: {mejor_score:.4f})")
    print(f"Modelo guardado en: {ruta_guardado}")
    
    return resultados, mejor_modelo_final

# 1. Entrenar y_arousal guardar el modelo
resultados, mejor_modelo = entrenar_evaluar_modelos(X, y_arousal, caracteristicas, 'modelo_arousal.joblib')

# 2. Imprimir resultados
for modelo, metricas in resultados.items():
    print(f"\nModelo: {modelo}")
    print(f"R2: {metricas['R2']:.4f}")
    print(f"RMSE: {metricas['RMSE']:.4f}")
    print(f"MAE: {metricas['MAE']:.4f}")
    print(f"Mejores parámetros: {metricas['Mejores_Parametros']}")



Mejor modelo: RandomForest (R2: 0.8150)
Modelo guardado en: modelo_arousal.joblib

Modelo: Ridge
R2: 0.5114
RMSE: 0.2005
MAE: 0.1585
Mejores parámetros: {'alpha': 10.0}

Modelo: Lasso
R2: 0.1817
RMSE: 0.2595
MAE: 0.2163
Mejores parámetros: {'alpha': 0.1}

Modelo: ElasticNet
R2: 0.4061
RMSE: 0.2211
MAE: 0.1790
Mejores parámetros: {'alpha': 0.1, 'l1_ratio': 0.2}

Modelo: RandomForest
R2: 0.8150
RMSE: 0.1234
MAE: 0.0884
Mejores parámetros: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}

Modelo: GradientBoosting
R2: 0.6705
RMSE: 0.1647
MAE: 0.1285
Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}

Modelo: XGBoost
R2: 0.6690
RMSE: 0.1650
MAE: 0.1285
Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}


Tratamiento de datos Features y valence

In [15]:
#Lectura y visualización archivo valence
df = pd.read_csv('valence.csv')
df

song_id  sample_15000ms  sample_15500ms  sample_16000ms  sample_16500ms  \
0           2       -0.073341       -0.074661       -0.074077       -0.078154   
1           3       -0.189702       -0.187765       -0.183740       -0.188761   
2           4        0.154286        0.148063        0.146956        0.151180   
3           5        0.149053        0.148260        0.147369        0.146956   
4           7        0.342816        0.345193        0.332023        0.328887   
...       ...             ...             ...             ...             ...   
1797     2054        0.088000        0.088000        0.086000        0.080000   
1798     2055        0.128000        0.098000        0.080000        0.080000   
1799     2056        0.198000        0.182000        0.164000        0.156000   
1800     2057       -0.071667       -0.061667       -0.076667       -0.078333   
1801     2058       -0.152000       -0.154000       -0.160000       -0.162000   

      sample_17000ms  sample_17500ms  sample_18000ms  sample_18500ms  \
0          -0.081588       -0.080873       -0.083611       -0.082145   
1          -0.193704       -0.190834       -0.189041       -0.197554   
2           0.135942        0.076832        0.084487        0.088513   
3           0.147857        0.146927        0.145651        0.145795   
4           0.327603        0.329011        0.327883        0.334822   
...              ...             ...             ...             ...   
1797        0.070000        0.068000        0.076000        0.078000   
1798        0.088000        0.088000        0.086000        0.078000   
1799        0.148000        0.142000        0.144000        0.146000   
1800       -0.081667       -0.046667       -0.046667       -0.068333   
1801       -0.158000       -0.162000       -0.166000       -0.164000   

      sample_19000ms  sample_19500ms  sample_20000ms  sample_20500ms  \
0          -0.081632       -0.083874       -0.085105       -0.087188   
1          -0.202871       -0.209757       -0.212038       -0.206937   
2           0.090944        0.085544        0.100404        0.179465   
3           0.146420        0.152565        0.153315        0.147846   
4           0.328447        0.344121        0.337617        0.339964   
...              ...             ...             ...             ...   
1797        0.074000        0.074000        0.074000        0.070000   
1798        0.068000        0.066000        0.066000        0.052000   
1799        0.160000        0.168000        0.178000        0.174000   
1800       -0.113333       -0.133333       -0.145000       -0.151667   
1801       -0.172000       -0.160000       -0.138000       -0.126000   

      sample_21000ms  sample_21500ms  sample_22000ms  sample_22500ms  \
0          -0.088078       -0.088742       -0.089806       -0.091019   
1          -0.206569       -0.215245       -0.225404       -0.232054   
2           0.183864        0.162215        0.136317        0.092502   
3           0.138537        0.137735        0.137825        0.156450   
4           0.329331        0.334279        0.335485        0.342974   
...              ...             ...             ...             ...   
1797        0.090000        0.090000        0.090000        0.090000   
1798        0.048000        0.042000        0.046000        0.054000   
1799        0.168000        0.160000        0.148000        0.162000   
1800       -0.158333       -0.161667       -0.165000       -0.165000   
1801       -0.120000       -0.140000       -0.180000       -0.194000   

      sample_23000ms  sample_23500ms  sample_24000ms  sample_24500ms  \
0          -0.092365       -0.093924       -0.095489       -0.097135   
1          -0.231102       -0.228758       -0.235275       -0.242087   
2           0.081918        0.096085        0.089629        0.090773   
3           0.204919        0.208784        0.239129        0.203294   
4           0.345424        0.341366        0.334168        0.347387   
...   

In [16]:
# Mantener todas las columnas hasta sample_44500ms (inclusive)
valence = df.loc[:, :'sample_44500ms']
valence.head()

,song_id,sample_15000ms,sample_15500ms,sample_16000ms,sample_16500ms,sample_17000ms,sample_17500ms,sample_18000ms,sample_18500ms,sample_19000ms,sample_19500ms,sample_20000ms,sample_20500ms,sample_21000ms,sample_21500ms,sample_22000ms,sample_22500ms,sample_23000ms,sample_23500ms,sample_24000ms,sample_24500ms,sample_25000ms,sample_25500ms,sample_26000ms,sample_26500ms,sample_27000ms,sample_27500ms,sample_28000ms,sample_28500ms,sample_29000ms,sample_29500ms,sample_30000ms,sample_30500ms,sample_31000ms,sample_31500ms,sample_32000ms,sample_32500ms,sample_33000ms,sample_33500ms,sample_34000ms,sample_34500ms,sample_35000ms,sample_35500ms,sample_36000ms,sample_36500ms,sample_37000ms,sample_37500ms,sample_38000ms,sample_38500ms,sample_39000ms,sample_39500ms,sample_40000ms,sample_40500ms,sample_41000ms,sample_41500ms,sample_42000ms,sample_42500ms,sample_43000ms,sample_43500ms,sample_44000ms,sample_44500ms
0,2,-0.073341,-0.074661,-0.074077,-0.078154,-0.081588,-0.080873,-0.083611,-0.082145,-0.081632,-0.083874,-0.085105,-0.087188,-0.088078,-0.088742,-0.089806,-0.091019,-0.092365,-0.093924,-0.095489,-0.097135,-0.118034,-0.194191,-0.211173,-0.212976,-0.214670,-0.214924,-0.216802,-0.224368,-0.239330,-0.241585,-0.243941,-0.246843,-0.250692,-0.251579,-0.259747,-0.286162,-0.298050,-0.296910,-0.302182,-0.306542,-0.309204,-0.311504,-0.311533,-0.309935,-0.310999,-0.309862,-0.310272,-0.310057,-0.313098,-0.314328,-0.317376,-0.324846,-0.324680,-0.328112,-0.330134,-0.330134,-0.331164,-0.331769,-0.334287,-0.333846
1,3,-0.189702,-0.187765,-0.183740,-0.188761,-0.193704,-0.190834,-0.189041,-0.197554,-0.202871,-0.209757,-0.212038,-0.206937,-0.206569,-0.215245,-0.225404,-0.232054,-0.231102,-0.228758,-0.235275,-0.242087,-0.242565,-0.238323,-0.235547,-0.260169,-0.283524,-0.287248,-0.285436,-0.290708,-0.299718,-0.298106,-0.297120,-0.299704,-0.304227,-0.301861,-0.297445,-0.294633,-0.304702,-0.310677,-0.309332,-0.307828,-0.314455,-0.321622,-0.310536,-0.310192,-0.317034,-0.314388,-0.316794,-0.320664,-0.316467,-0.312543,-0.302453,-0.313982,-0.305432,-0.294850,-0.282940,-0.276367,-0.279427,-0.278662,-0.274248,-0.270162
2,4,0.154286,0.148063,0.146956,0.151180,0.135942,0.076832,0.084487,0.088513,0.090944,0.085544,0.100404,0.179465,0.183864,0.162215,0.136317,0.092502,0.081918,0.096085,0.089629,0.090773,0.099315,0.098499,0.100844,0.100649,0.098971,0.110439,0.116545,0.128038,0.133658,0.133888,0.134824,0.149676,0.170748,0.183955,0.194449,0.195255,0.194667,0.194214,0.195980,0.196551,0.201931,0.201426,0.200411,0.198420,0.197428,0.203620,0.206903,0.210580,0.213040,0.215211,0.217544,0.217568,0.218895,0.222275,0.190338,0.173086,0.174892,0.179884,0.182710,0.179332
3,5,0.149053,0.148260,0.147369,0.146956,0.147857,0.146927,0.145651,0.145795,0.146420,0.152565,0.153315,0.147846,0.138537,0.137735,0.137825,0.156450,0.204919,0.208784,0.239129,0.203294,0.142848,0.129963,0.125613,0.122065,0.121132,0.120463,0.121303,0.121303,0.121635,0.122608,0.122939,0.126212,0.129973,0.131580,0.132089,0.132173,0.132173,0.132173,0.132173,0.132173,0.132173,0.132173,0.132493,0.132608,0.129884,0.130001,0.132583,0.132614,0.130502,0.129999,0.132608,0.133477,0.133477,0.134290,0.133555,0.134592,0.134347,0.133955,0.133477,0.133477
4,7,0.342816,0.345193,0.332023,0.328887,0.327603,0.329011,0.327883,0.334822,0.328447,0.344121,0.337617,0.339964,0.329331,0.334279,0.335485,0.342974,0.345424,0.341366,0.334168,0.347387,0.347424,0.351215,0.352030,0.344706,0.353899,0.347019,0.345312,0.329848,0.325029,0.335758,0.339655,0.319205,0.334188,0.335342,0.323406,0.314509,0.322830,0.334673,0.324579,0.331797,0.308347,0.349843,0.339804,0.325933,0.328986,0.338838,0.338114,0.326702,0.315049,0.345829,0.349402,0.327612,0.330913,0.331320,0.301550,0.295276,0.295031,0.319244,0.316518,0.321731


In [17]:
#Transponer columnas sample

# Identificar las columnas de sample
sample_columns = [col for col in valence.columns if col.startswith("sample_")]

# Usar melt para transponer las columnas de sample en una sola columna llamada "valence"
df_melted = valence.melt(id_vars=["song_id"],
                    value_vars=sample_columns,
                    var_name="sample",
                    value_name="valence")

# Extraer solo el número del sample (por ejemplo, de "sample_1500" a "1500")
df_melted["sample"] = df_melted["sample"].str.extract(r'(\d+)').astype(int)

# Ordenar por "song_id" y luego por "sample" para asegurar que los datos estén bien organizados
df_melted_valence = df_melted.sort_values(by=["song_id", "sample"]).reset_index(drop=True)

# Mostrar el resultado para verificar

In [18]:
#Visualización del dataset transpuesto
df_melted_valence

,song_id,sample,valence
0,2,15000,-0.073341
1,2,15500,-0.074661
2,2,16000,-0.074077
3,2,16500,-0.078154
4,2,17000,-0.081588
...,...,...,...
108115,2058,42500,-0.218000
108116,2058,43000,-0.222000
108117,2058,43500,-0.226000
108118,2058,44000,-0.228000


In [19]:
#Se realiza el merge del dataframe features y valence
df_fvalence = pd.merge(features, df_melted_valence, on=['song_id'], how='inner')
df_fvalence

,frameTime,F0final_sma_stddev,F0final_sma_amean,voicingFinalUnclipped_sma_stddev,voicingFinalUnclipped_sma_amean,jitterLocal_sma_stddev,jitterLocal_sma_amean,jitterDDP_sma_stddev,jitterDDP_sma_amean,shimmerLocal_sma_stddev,shimmerLocal_sma_amean,logHNR_sma_stddev,logHNR_sma_amean,audspec_lengthL1norm_sma_stddev,audspec_lengthL1norm_sma_amean,audspecRasta_lengthL1norm_sma_stddev,audspecRasta_lengthL1norm_sma_amean,pcm_RMSenergy_sma_stddev,pcm_RMSenergy_sma_amean,pcm_zcr_sma_stddev,pcm_zcr_sma_amean,audSpec_Rfilt_sma[0]_stddev,audSpec_Rfilt_sma[0]_amean,audSpec_Rfilt_sma[1]_stddev,audSpec_Rfilt_sma[1]_amean,audSpec_Rfilt_sma[2]_stddev,audSpec_Rfilt_sma[2]_amean,audSpec_Rfilt_sma[3]_stddev,audSpec_Rfilt_sma[3]_amean,audSpec_Rfilt_sma[4]_stddev,audSpec_Rfilt_sma[4]_amean,audSpec_Rfilt_sma[5]_stddev,audSpec_Rfilt_sma[5]_amean,audSpec_Rfilt_sma[6]_stddev,audSpec_Rfilt_sma[6]_amean,audSpec_Rfilt_sma[7]_stddev,audSpec_Rfilt_sma[7]_amean,audSpec_Rfilt_sma[8]_stddev,audSpec_Rfilt_sma[8]_amean,audSpec_Rfilt_sma[9]_stddev,audSpec_Rfilt_sma[9]_amean,audSpec_Rfilt_sma[10]_stddev,audSpec_Rfilt_sma[10]_amean,audSpec_Rfilt_sma[11]_stddev,audSpec_Rfilt_sma[11]_amean,audSpec_Rfilt_sma[12]_stddev,audSpec_Rfilt_sma[12]_amean,audSpec_Rfilt_sma[13]_stddev,audSpec_Rfilt_sma[13]_amean,audSpec_Rfilt_sma[14]_stddev,audSpec_Rfilt_sma[14]_amean,audSpec_Rfilt_sma[15]_stddev,audSpec_Rfilt_sma[15]_amean,audSpec_Rfilt_sma[16]_stddev,audSpec_Rfilt_sma[16]_amean,audSpec_Rfilt_sma[17]_stddev,audSpec_Rfilt_sma[17]_amean,audSpec_Rfilt_sma[18]_stddev,audSpec_Rfilt_sma[18]_amean,audSpec_Rfilt_sma[19]_stddev,audSpec_Rfilt_sma[19]_amean,audSpec_Rfilt_sma[20]_stddev,audSpec_Rfilt_sma[20]_amean,audSpec_Rfilt_sma[21]_stddev,audSpec_Rfilt_sma[21]_amean,audSpec_Rfilt_sma[22]_stddev,audSpec_Rfilt_sma[22]_amean,audSpec_Rfilt_sma[23]_stddev,audSpec_Rfilt_sma[23]_amean,audSpec_Rfilt_sma[24]_stddev,audSpec_Rfilt_sma[24]_amean,audSpec_Rfilt_sma[25]_stddev,audSpec_Rfilt_sma[25]_amean,pcm_fftMag_fband250-650_sma_stddev,pcm_fftMag_fband250-650_sma_amean,pcm_fftMag_fband1000-4000_sma_stddev,pcm_fftMag_fband1000-4000_sma_amean,pcm_fftMag_spectralRollOff25.0_sma_stddev,pcm_fftMag_spectralRollOff25.0_sma_amean,pcm_fftMag_spectralRollOff50.0_sma_stddev,pcm_fftMag_spectralRollOff50.0_sma_amean,pcm_fftMag_spectralRollOff75.0_sma_stddev,pcm_fftMag_spectralRollOff75.0_sma_amean,pcm_fftMag_spectralRollOff90.0_sma_stddev,pcm_fftMag_spectralRollOff90.0_sma_amean,pcm_fftMag_spectralFlux_sma_stddev,pcm_fftMag_spectralFlux_sma_amean,pcm_fftMag_spectralCentroid_sma_stddev,pcm_fftMag_spectralCentroid_sma_amean,pcm_fftMag_spectralEntropy_sma_stddev,pcm_fftMag_spectralEntropy_sma_amean,pcm_fftMag_spectralVariance_sma_stddev,pcm_fftMag_spectralVariance_sma_amean,pcm_fftMag_spectralSkewness_sma_stddev,pcm_fftMag_spectralSkewness_sma_amean,pcm_fftMag_spectralKurtosis_sma_stddev,pcm_fftMag_spectralKurtosis_sma_amean,pcm_fftMag_spectralSlope_sma_stddev,pcm_fftMag_spectralSlope_sma_amean,pcm_fftMag_psySharpness_sma_stddev,pcm_fftMag_psySharpness_sma_amean,pcm_fftMag_spectralHarmonicity_sma_stddev,pcm_fftMag_spectralHarmonicity_sma_amean,pcm_fftMag_mfcc_sma[1]_stddev,pcm_fftMag_mfcc_sma[1]_amean,pcm_fftMag_mfcc_sma[2]_stddev,pcm_fftMag_mfcc_sma[2]_amean,pcm_fftMag_mfcc_sma[3]_stddev,pcm_fftMag_mfcc_sma[3]_amean,pcm_fftMag_mfcc_sma[4]_stddev,pcm_fftMag_mfcc_sma[4]_amean,pcm_fftMag_mfcc_sma[5]_stddev,pcm_fftMag_mfcc_sma[5]_amean,pcm_fftMag_mfcc_sma[6]_stddev,pcm_fftMag_mfcc_sma[6]_amean,pcm_fftMag_mfcc_sma[7]_stddev,pcm_fftMag_mfcc_sma[7]_amean,pcm_fftMag_mfcc_sma[8]_stddev,pcm_fftMag_mfcc_sma[8]_amean,pcm_fftMag_mfcc_sma[9]_stddev,pcm_fftMag_mfcc_sma[9]_amean,pcm_fftMag_mfcc_sma[10]_stddev,pcm_fftMag_mfcc_sma[10]_amean,pcm_fftMag_mfcc_sma[11]_stddev,pcm_fftMag_mfcc_sma[11]_amean,pcm_fftMag_mfcc_sma[12]_stddev,pcm_fftMag_mfcc_sma[12]_amean,pcm_fftMag_mfcc_sma[13]_stddev,pcm_fftMag_mfcc_sma[13]_amean,pcm_fftMag_mfcc_sma[14]_stddev,pcm_fftMag_mfcc_sma[14]_amean,F0final_sma_de_stddev,F0final_sma_de_amean,voicing

In [20]:
# Tomamos la muestra para hacer el análisis manejable
sample_size = 100000
df_s_valence = df_fvalence.sample(n=sample_size, random_state=42)

In [21]:
def analizar_caracteristicas(X, y_valence, umbral_varianza=0.95):
    # Estandarizar características
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Análisis PCA para determinar número óptimo de componentes
    pca = PCA()
    pca.fit(X_scaled)
    
    # Calcular varianza acumulada
    varianza_acumulada = np.cumsum(pca.explained_variance_ratio_)
    n_componentes = np.argmax(varianza_acumulada >= umbral_varianza) + 1
    
    # Selección usando f_regression
    selector = SelectKBest(score_func=f_regression)
    selector.fit(X_scaled, y_valence)
    
    # Calcular importancia mutua
    mi_scores = mutual_info_regression(X_scaled, y_valence)
    
    # Combinar resultados
    resultados = pd.DataFrame({
        'Característica': X.columns,
        'F_Score': selector.scores_,
        'MI_Score': mi_scores
    })
    
    # Normalizar scores
    resultados['F_Score_Norm'] = (resultados['F_Score'] - resultados['F_Score'].min()) / (resultados['F_Score'].max() - resultados['F_Score'].min())
    resultados['MI_Score_Norm'] = (resultados['MI_Score'] - resultados['MI_Score'].min()) / (resultados['MI_Score'].max() - resultados['MI_Score'].min())
    
    # Score combinado
    resultados['Score_Combinado'] = (resultados['F_Score_Norm'] + resultados['MI_Score_Norm']) / 2
    
    # Ordenar por score combinado
    resultados_ordenados = resultados.sort_values('Score_Combinado', ascending=False)
    
    # Determinar número de características basado en PCA
    caracteristicas_seleccionadas = resultados_ordenados.head(n_componentes)
    
    return caracteristicas_seleccionadas, n_componentes, varianza_acumulada

def evaluar_modelo(X, y_valence, caracteristicas_seleccionadas):
    # Usar solo las características seleccionadas
    X_reducido = X[caracteristicas_seleccionadas['Característica']]
    
    # Split y_valence entrenamiento
    X_train, X_test, y_train, y_test = train_test_split(X_reducido, y_valence, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Evaluar
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    return r2, rmse, model

# Ejemplo de uso
# df = tu_dataframe
X = df_s_valence.drop(['frameTime','song_id','sample'], axis=1)  # Ajusta según tus columnas
y_valence = X['valence'] if 'valence' in X.columns else None
X = X.drop(['valence'], axis=1)

# Análisis de características
umbral_varianza = 0.95
caracteristicas, n_componentes, varianza = analizar_caracteristicas(X, y_valence, umbral_varianza)
print(f"Número óptimo de características para mantener {umbral_varianza*100}% de la varianza: {n_componentes}")
print(f"Número óptimo de características para mantener {umbral_varianza*100}% de la varianza: {n_componentes}")

# Evaluación del modelo
r2, rmse, modelo = evaluar_modelo(X, y_valence, caracteristicas)
print(f"\nRendimiento del modelo:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")

# Mostrar top características
print("\nTop características seleccionadas:")
print(caracteristicas[['Característica', 'Score_Combinado']])

Número óptimo de características para mantener 95.0% de la varianza: 87
Número óptimo de características para mantener 95.0% de la varianza: 87

Rendimiento del modelo:
R² Score: 0.2978
RMSE: 0.1978

Top características seleccionadas:
                                   Característica  Score_Combinado
142            audspec_lengthL1norm_sma_de_stddev         0.980651
103                  pcm_fftMag_mfcc_sma[1]_amean         0.940722
220     pcm_fftMag_spectralVariance_sma_de_stddev         0.777545
212  pcm_fftMag_spectralRollOff90.0_sma_de_stddev         0.708650
90         pcm_fftMag_spectralVariance_sma_stddev         0.696709
..                                            ...              ...
105                  pcm_fftMag_mfcc_sma[2]_amean         0.207329
152                audSpec_Rfilt_sma_de[1]_stddev         0.203107
62                   audSpec_Rfilt_sma[21]_stddev         0.201262
150                audSpec_Rfilt_sma_de[0]_stddev         0.200988
190               audSpec_Rf

In [22]:
def entrenar_evaluar_modelos(X, y_valence, caracteristicas_seleccionadas, ruta_guardado='mejor_modelo.joblib'):
    # Usar características seleccionadas
    X_reducido = X[caracteristicas_seleccionadas['Característica']]
    
    # Split de datos
    X_train, X_test, y_train, y_test = train_test_split(
        X_reducido, y_valence, test_size=0.2, random_state=42
    )
    
    # Escalado
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Guardar el scaler para uso futuro
    joblib.dump(scaler, 'scaler.joblib')
    
    # Definir modelos
    modelos = {
        'Ridge': Ridge(),
        'Lasso': Lasso(),
        'ElasticNet': ElasticNet(),
        'RandomForest': RandomForestRegressor(),
        'GradientBoosting': GradientBoostingRegressor(),
        'XGBoost': xgb.XGBRegressor()
    }
    
    # Parámetros para GridSearch
    parametros = {
        'Ridge': {'alpha': [0.1, 1.0, 10.0]},
        'Lasso': {'alpha': [0.1, 1.0, 10.0]},
        'ElasticNet': {'alpha': [0.1, 1.0], 'l1_ratio': [0.2, 0.5, 0.8]},
        'RandomForest': {
            'n_estimators': [100, 200],
            'max_depth': [10, 20],
            'min_samples_split': [2, 5]
        },
        'GradientBoosting': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5]
        },
        'XGBoost': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5]
        }
    }
    
    resultados = {}
    mejor_score = float('-inf')
    mejor_modelo_final = None
    
    for nombre, modelo in modelos.items():
        # GridSearch con validación cruzada
        grid = GridSearchCV(
            modelo, 
            parametros[nombre], 
            cv=3, 
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        
        grid.fit(X_train_scaled, y_train)
        
        # Predicciones
        y_pred = grid.predict(X_test_scaled)
        
        # Métricas
        r2 = r2_score(y_test, y_pred)
        resultados[nombre] = {
            'R2': r2,
            'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
            'MAE': mean_absolute_error(y_test, y_pred),
            'Mejores_Parametros': grid.best_params_
        }
        
        # Actualizar mejor modelo basado en R2
        if r2 > mejor_score:
            mejor_score = r2
            mejor_modelo_final = grid.best_estimator_
            mejor_nombre = nombre
    
    # Guardar el mejor modelo
    joblib.dump(mejor_modelo_final, ruta_guardado)
    
    # Guardar información sobre las características seleccionadas
    joblib.dump(caracteristicas_seleccionadas['Característica'].tolist(), 'caracteristicas_seleccionadas.joblib')
    
    print(f"\nMejor modelo: {mejor_nombre} (R2: {mejor_score:.4f})")
    print(f"Modelo guardado en: {ruta_guardado}")
    
    return resultados, mejor_modelo_final

# 1. Entrenar y_valence guardar el modelo
resultados, mejor_modelo = entrenar_evaluar_modelos(X, y_valence, caracteristicas, 'modelo_valence.joblib')

# 2. Imprimir resultados
for modelo, metricas in resultados.items():
    print(f"\nModelo: {modelo}")
    print(f"R2: {metricas['R2']:.4f}")
    print(f"RMSE: {metricas['RMSE']:.4f}")
    print(f"MAE: {metricas['MAE']:.4f}")
    print(f"Mejores parámetros: {metricas['Mejores_Parametros']}")


Mejor modelo: RandomForest (R2: 0.7401)
Modelo guardado en: modelo_valence.joblib

Modelo: Ridge
R2: 0.2978
RMSE: 0.1978
MAE: 0.1585
Mejores parámetros: {'alpha': 1.0}

Modelo: Lasso
R2: -0.0000
RMSE: 0.2360
MAE: 0.1946
Mejores parámetros: {'alpha': 0.1}

Modelo: ElasticNet
R2: 0.1667
RMSE: 0.2155
MAE: 0.1747
Mejores parámetros: {'alpha': 0.1, 'l1_ratio': 0.2}

Modelo: RandomForest
R2: 0.7401
RMSE: 0.1203
MAE: 0.0888
Mejores parámetros: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}

Modelo: GradientBoosting
R2: 0.5276
RMSE: 0.1622
MAE: 0.1286
Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}

Modelo: XGBoost
R2: 0.5302
RMSE: 0.1618
MAE: 0.1281
Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
